In [ ]:
validate_dataset = AmericanExpressPreprocessedProfileTimeSeriesDataset("./amex_preprocessed.npz", test=True)
validate_dataloader = DataLoader(validate_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
def validate_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    f1 = torchmetrics.F1Score().to(device)
    precision = torchmetrics.Precision(num_classes=2).to(device)
    recall = torchmetrics.Recall(num_classes=2).to(device)

    all_preds = np.array([])
    all_ys = np.array([])

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            all_preds = np.append(all_preds, pred.view(-1).cpu().detach(), axis=0)
            all_ys = np.append(all_ys, y.view(-1).cpu().detach(), axis=0)
            pred = torch.round(pred)
            f1.update(torch.tensor(pred, dtype=torch.int16).view(-1), torch.tensor(y, dtype=torch.int16).view(-1))
            precision.update(torch.tensor(pred, dtype=torch.int16).view(-1), torch.tensor(y, dtype=torch.int16).view(-1))
            recall.update(torch.tensor(pred, dtype=torch.int16).view(-1), torch.tensor(y, dtype=torch.int16).view(-1))

            correct += (pred == y).type(torch.float).sum().item()


    test_loss /= num_batches
    correct /= size

    f1_score = f1.compute()
    precision = precision.compute()
    recall = recall.compute()

    amex = get_amex(all_preds, all_ys)

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print("precision", precision)
    print("recall", recall)
    print("amex", amex)
    print("f1", f1_score)

In [ ]:
validate_loop(validate_dataloader, model, loss_fn)